In [7]:
"""
ZRA Column Mapper Script
========================
This script maps ZRA columns from product_import_template.xlsx (df_clean) 
to ProductTemplate.xlsx (df_raw) using Odoo's x_* field naming convention.

The resulting file can be used to update products in Odoo 18.
"""

import pandas as pd
import numpy as np

# Load the data files
df_clean = pd.read_excel('/Users/malvernbright/Desktop/ML/product_import_template.xlsx')
df_raw = pd.read_excel('/Users/malvernbright/Desktop/ML/ProductTemplate.xlsx')

print(f"Loaded df_clean: {len(df_clean)} products, {len(df_clean.columns)} columns")
print(f"Loaded df_raw: {len(df_raw)} products, {len(df_raw.columns)} columns")

# Column mapping: df_clean (human-readable) -> df_raw (Odoo x_* fields)
column_mapping = {
    'Item Classification': 'x_itemCls',
    'Item Classification Code(zra)': 'x_itemClsCd',
    'Origin Place Code (Nation)': 'x_orgnNatCd',
    'Origin Place (Nation)': 'x_orgnNat',
    'Export Nation Code': 'x_exptNatCd',
    'Export Nation': 'x_exptNat',
    'Packaging Unit Code (zra)': 'x_pkgUnitCd',
    'Packaging Unit': 'x_pkgUnit',
    'VAT Category Code': 'x_vatCatCd',
    'ZRA Product Type': 'x_itemTyCd',
    'Quantity Unit Code(zra)': 'x_qtyUnitCd',
    'ZRA Unit of Measure': 'x_UnitCd',
    'ZRA Purchase Unit of Measure': 'x_PacCd',
    'Insurance Applicable (Y/N)': 'x_isrcAplcbYn',
    'Indication of whether an item has rental or not': 'x_rentalYn',
    'Indication of whether an item has service charge': 'x_svcChargeYn',
    'Used/UnUsed': 'x_useYn',
    'ZRA Modify Y/N': 'x_ZRAModYn',
    'tlCatCd': 'x_tlCatCd',
    'ExciseCatCd': 'x_exciseTxCatCd',
    'Destination Country Code': 'x_export_country_id',
    'Is Import Item?': 'x_is_import_item',
    'Declaration Date': 'x_dclDe',
    'Import Item Status Code': 'x_imptItemsttsCd',
    'Manufacturer item code for MTV product': 'x_manufacturerItemCd',
    'Manufacturer TPIN for MTV product': 'x_manufactuterTpin',
    'RRP': 'x_rrp',
    'Product name': 'name',
    'Barcode': 'barcode',
    'Internal Reference': 'default_code',
    'Sales Price': 'list_price',
    'Cost': 'standard_price'
}

print(f"\nColumn mapping defined: {len(column_mapping)} mappings")

# Rename columns in df_clean to match Odoo x_* field names
df_clean_renamed = df_clean.rename(columns=column_mapping)

# Identify ZRA columns (x_*) from df_clean_renamed
zra_columns_in_clean = [col for col in df_clean_renamed.columns if col.startswith('x_')]
zra_columns_in_raw = [col for col in df_raw.columns if col.startswith('x_')]

print(f"\nZRA columns in df_clean (renamed): {len(zra_columns_in_clean)}")
print(f"ZRA columns in df_raw: {len(zra_columns_in_raw)}")

# Check product matching
print(f"\nProducts in df_raw: {len(df_raw)}")
print(f"Products in df_clean: {len(df_clean_renamed)}")

# Check for matching product names
common_names = set(df_raw['name'].dropna()) & set(df_clean_renamed['name'].dropna())
print(f"Products matching by name: {len(common_names)}")

# Select columns to merge (only the ZRA x_* columns we need)
cols_to_merge = ['name'] + zra_columns_in_clean
df_clean_subset = df_clean_renamed[cols_to_merge].copy()

# Merge the ZRA columns from df_clean into df_raw based on product name
# Using left join to keep all products in df_raw
df_merged = df_raw.merge(
    df_clean_subset, 
    on='name', 
    how='left', 
    suffixes=('', '_new')
)

print(f"\nMerged dataframe shape: {df_merged.shape}")

# Update existing ZRA columns with new values from df_clean where available
for col in zra_columns_in_clean:
    new_col = f'{col}_new'
    if new_col in df_merged.columns:
        # Update original column with new values where they exist
        df_merged[col] = df_merged[new_col].combine_first(df_merged[col])
        # Drop the _new suffix column
        df_merged = df_merged.drop(columns=[new_col])

print(f"Final merged dataframe shape: {df_merged.shape}")

# Save the updated dataframe to a new Excel file for Odoo import
output_path = '/Users/malvernbright/Desktop/ML/ProductTemplate_UpdatedUpdated.xlsx'
df_merged.to_excel(output_path, index=False)

print(f"\n{'='*60}")
print(f"Updated product template saved to: {output_path}")
print(f"Total products: {len(df_merged)}")
print(f"Total columns: {len(df_merged.columns)}")
print(f"{'='*60}")

# Summary of ZRA fields now in the updated template
zra_cols_final = [col for col in df_merged.columns if col.startswith('x_')]
print(f"\nZRA fields (x_*) now present in the updated template ({len(zra_cols_final)} fields):")
for i, col in enumerate(zra_cols_final, 1):
    print(f"  {i}. {col}")


Loaded df_clean: 2167 products, 35 columns
Loaded df_raw: 993 products, 71 columns

Column mapping defined: 32 mappings

ZRA columns in df_clean (renamed): 27
ZRA columns in df_raw: 52

Products in df_raw: 993
Products in df_clean: 2167
Products matching by name: 925

Merged dataframe shape: (999, 98)
Final merged dataframe shape: (999, 71)


/var/folders/77/_9nb3lq52l77gj4r60ff54m80000gn/T/ipykernel_96570/1726945200.py:96: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merged[col] = df_merged[new_col].combine_first(df_merged[col])
/var/folders/77/_9nb3lq52l77gj4r60ff54m80000gn/T/ipykernel_96570/1726945200.py:96: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_merged[col] = df_merged[new_col].combine_first(df_merged[col])



Updated product template saved to: /Users/malvernbright/Desktop/ML/ProductTemplate_UpdatedUpdated.xlsx
Total products: 999
Total columns: 71

ZRA fields (x_*) now present in the updated template (52 fields):
  1. x_studio_ic
  2. x_agntNm
  3. x_btchNo
  4. x_dclDe
  5. x_dclNo
  6. x_dclRefNum
  7. x_export_country_id
  8. x_exciseTxCatCd
  9. x_exptCd
  10. x_exptNat
  11. x_exptNatCd
  12. x_totWt
  13. x_hsCd
  14. x_imptItemsttsCd
  15. x_rentalYn
  16. x_svcChargeYn
  17. x_isrcAplcbYn
  18. x_invcFcurExcrt
  19. x_invcFcurAmt
  20. x_invcFcurCd
  21. x_iplCatCd
  22. x_is_import_item
  23. x_is_processed
  24. x_is_registered
  25. x_is_rrp_item
  26. x_is_success
  27. x_itemCls
  28. x_ClassCd
  29. x_itemClsCd
  30. x_itemCd
  31. x_itemSeq
  32. x_manufacturerItemCd
  33. x_manufactuterTpin
  34. x_netWt
  35. x_orgnNat
  36. x_orgnCd
  37. x_orgnNatCd
  38. x_pkgUnit
  39. x_PacCd
  40. x_pkgUnitCd
  41. x_UnitCd
  42. x_qtyUnitCd
  43. x_registered_item_code
  44. x_rrp
 

In [11]:
df_new = pd.read_excel("/Users/malvernbright/Desktop/ML/ProductTemplate_UpdatedUpdated.xlsx")
df_new.head()

,id,activity_exception_decoration,combo_ids,standard_price,is_favorite,virtual_available,x_studio_ic,default_code,name,qty_available,...,x_taxTyCd,x_tlCatCd,x_useYn,x_vatCatCd,x_ZRAModYn,x_itemTyCd,id.1,zra_purchase_uom_id,zra_uom_id,barcode
0,__export__.product_template_63201_3e20ff9a,NaN,NaN,0.0,False,0,NaN,NaN,NAOMI SHOWER GEL OUD & ROSE 1L,0,...,NaN,2,Y,A,Y,2,__export__.product_template_63201_3e20ff9a,NaN,NaN,6.290361e+12
1,__export__.product_template_63984_e7c25b74,NaN,NaN,0.0,False,0,NaN,NaN,AL SHAFAA NATURAL HONEY 500G,0,...,NaN,2,Y,A,Y,2,__export__.product_template_63984_e7c25b74,NaN,NaN,8.906125e+12
2,__export__.product_template_65187_df2f0e26,NaN,NaN,0.0,False,0,Unclassified product [4332255500],NaN,AL-RISHENG LORD PERFUME 100ML,0,...,A,2,Y,A,Y,2,__export__.product_template_65187_df2f0e26,Pieces/item [Number],Pieces/item [Number],6.927525e+12
3,__export__.product_template_63202_6c4e0140,NaN,NaN,0.0,False,0,Dried cut santa fe rose [10402765],NaN,AL-RISHENG LORD PERFUME 100ML,0,...,A,2,Y,A,Y,2,__export__.product_template_63202_6c4e0140,Each,Each,6.927525e+12
4,__export__.product_template_65097_ec35d219,NaN,NaN,0.0,False,0,NaN,NaN,ALOHA PO,0,...,NaN,NaN,Yes,A,Yes,NaN,__export__.product_template_65097_ec35d219,NaN,NaN,NaN


In [12]:
df3 = pd.read_excel('/Users/malvernbright/Desktop/ML/product_import_template.xlsx')
df3.head()

,Product name,Sales Price,Barcode,Cost,Internal Reference,Item Classification,Item Classification Code(zra),Origin Place Code (Nation),Origin Place (Nation),Export Nation Code,...,tlCatCd,ExciseCatCd,Destination Country Code,Is Import Item?,Declaration Date,Import Item Status Code,Manufacturer item code for MTV product,Manufacturer TPIN for MTV product,RRP,Point of Sale Category
0,NAOMI SHOWER GEL OUD & ROSE 1L,78.0,6.290361e+12,78.0,NaN,4.332256e+09,Unclassified product,ZM,ZAMBIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GROCERY
1,AL-RISHENG LORD PERFUME 100ML,29.0,6.927525e+12,29.0,NaN,4.332256e+09,Unclassified product,ZM,ZAMBIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GROCERY
2,ARMAF BODY SPRAY CANDEE 200ML,53.0,6.295200e+12,53.0,NaN,4.332256e+09,Unclassified product,ZM,ZAMBIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GROCERY
3,ARMAF BODY SPRAY TRES NUIT 200ML,53.0,6.085010e+12,53.0,NaN,4.332256e+09,Unclassified product,ZM,ZAMBIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GROCERY
4,ARMAF BODY SPRAY VANITY FEMME 200ML,53.0,6.085010e+12,53.0,NaN,4.332256e+09,Unclassified product,ZM,ZAMBIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GROCERY
